# Calculating Cracked Properties

This example demonstrates how to obtain and display the cracked area properties. We start by importing the necessary modules.

In [ ]:
import numpy as np
from sectionproperties.pre.library import concrete_tee_section

from concreteproperties import (
    Concrete,
    ConcreteLinear,
    ConcreteSection,
    RectangularStressBlock,
    SteelBar,
    SteelElasticPlastic,
)

## Assign Materials

The materials used in this example will be 40 MPa concrete and 500 MPa steel, specified in accordance with AS 3600:2018.

In [ ]:
concrete = Concrete(
    name="40 MPa Concrete",
    density=2.4e-6,
    stress_strain_profile=ConcreteLinear(elastic_modulus=32.8e3),
    ultimate_stress_strain_profile=RectangularStressBlock(
        compressive_strength=40,
        alpha=0.79,
        gamma=0.87,
        ultimate_strain=0.003,
    ),
    flexural_tensile_strength=3.8,
    colour="lightgrey",
)

steel = SteelBar(
    name="500 MPa Steel",
    density=7.85e-6,
    stress_strain_profile=SteelElasticPlastic(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
    ),
    colour="grey",
)

## Create Geometry and Concrete Section

The section being analysed in this example is a 900D x 300W tee-beam with a 1200W x 200D slab. The reinforcement detailed is 6N16 top bars and 3N32 bottom bars, with 30 mm of clear cover. The geometry is generated using the *sectionproperties* concrete sections library.

In [ ]:
geom = concrete_tee_section(
    d=900,
    b=300,
    d_f=200,
    b_f=1200,
    dia_top=16,
    area_top=200,
    n_top=6,
    c_top=30,
    dia_bot=32,
    area_bot=800,
    n_bot=3,
    c_bot=30,
    conc_mat=concrete,
    steel_mat=steel,
)

conc_sec = ConcreteSection(geom)
conc_sec.plot_section()

## Calculate Cracked Properties

In this example we'll peform a cracked analysis for sagging ($\theta=0$) and hogging ($\theta=\pi$). To do this we call the ``calculate_cracked_properties()`` method:

In [ ]:
cracked_res_sag = conc_sec.calculate_cracked_properties()
cracked_res_hog = conc_sec.calculate_cracked_properties(theta=np.pi)

We can print the results of these analyses by calling the ``print_results`` method:

In [ ]:
cracked_res_sag.print_results()
cracked_res_hog.print_results()

## Transformed Cracked Properties

We can compute the transformed properties by calling the ``calculate_transformed_properties()`` method on the ``CrackedResults`` objects and also print these results:

In [ ]:
from concreteproperties.post import si_kn_m, si_n_mm

si_n_mm.radians = False  # show degrees

cracked_res_sag.calculate_transformed_properties(elastic_modulus=32.8e3)
cracked_res_hog.calculate_transformed_properties(elastic_modulus=32.8e3)

cracked_res_sag.print_results(units=si_kn_m)
cracked_res_hog.print_results(units=si_n_mm)

Specific results can be obtained by referencing the associated attribute in the ``CrackedResults`` object:

In [ ]:
cracking_moment = cracked_res_sag.m_cr
neutral_axis_depth = cracked_res_sag.d_nc
cracked_i = cracked_res_sag.iuu_cr

print(f"M_cr = {cracking_moment / 1e6:.2f} kN.m")
print(f"d_nc = {neutral_axis_depth:.2f} mm")
print(f"I_cr = {cracked_i:.3e} mm^4")

## Plot Cracked Geometries

Finally, the cracked geometry can be displayed by calling the ``plot_cracked_geometries()`` method:

In [ ]:
cracked_res_sag.plot_cracked_geometries(labels=[], cp=False, legend=False)

In [ ]:
cracked_res_hog.plot_cracked_geometries(labels=[], cp=False, legend=False)